**Desription**:
    
PB 

**NOTE:** CrowdVar Size Range

The following notebook is a revised version of the [Crowdvariant analysis notebook](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Machine%20Learning%5BAll%20Technologies%5D.ipynb). 

In the first part of this notebook, a random forest classifier was trained on crowdsourced[CrowdVariant](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Data%20Collection.ipynb) data for HG002 Deletions. 

In the second part of this notebook, the trained randfom forest classifier is then used to predict labels for a new set of randomly selected HG002 Deletions.

In [1]:
"""
Imports
"""
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



***
Load Data
***
Restructure features so that the training dataset and prediction dataset have all of the same features

In [17]:
### Import Data
df_crowd = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_300x.csv')
### Copy data in new dataframe as a later reference
df_crowd_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_300x.csv')

In [18]:
df_crowd.rename(columns={'size': 'Size'}, inplace=True)

In [19]:
df_crowd.head(3)

,chrom,start,end,sample,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,...,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label,CN0_prob,CN1_prob,CN2_prob
0,1,187464828,187466479,HG002,579.446154,17.934094,65.0,756.430769,163.879409,0.0,...,8,0.096911,0,0.0,0,0,1,0.00,0.91,0.09
1,1,33156824,33157000,HG002,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,...,2,0.221591,0,0.0,0,0,1,0.04,0.91,0.05
2,1,53594099,53595428,HG002,574.335366,18.613946,164.0,678.518293,139.056203,31.0,...,3,0.059443,0,0.0,0,0,0,0.96,0.04,0.00


In [20]:
# Read in HG002 DEL dataframe
HG002_pred = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_300x.csv')
HG002_pred_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_300x.csv')

In [21]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(c) - set(d)

{'CN0_prob', 'CN1_prob', 'CN2_prob', 'Label'}

In [ ]:
# a = ['chrom','start','end','sample','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_insertSizeScore_insertSizeScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','size','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','Label','CN0_prob','CN1_prob','CN2_prob']
# b = ['chrom','start','end','sample','id','type','SVtype','Size','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']

# set(b) - set(a)

In [22]:
### Training Set: Drop irrelevant columns and GT information
df_crowd.drop(['GTcons'], axis=1, inplace = True)
df_crowd.drop(['GTconflict'], axis=1, inplace = True)
df_crowd.drop(['GTsupp'], axis=1, inplace = True)
# df_crowd.drop('SVtype', axis=1)
# df_crowd.drop(['type'],axis=1)
df_crowd.drop(['start'],axis=1, inplace = True)
df_crowd.drop(['end'],axis=1, inplace = True)
df_crowd.drop(['chrom'],axis=1, inplace = True)
# df_crowd.drop('Size',axis=1)
df_crowd.drop(['CN0_prob'],axis=1, inplace = True)
df_crowd.drop(['CN1_prob'],axis=1, inplace = True)
df_crowd.drop(['CN2_prob'],axis=1, inplace = True)
# df_crowd.drop(['TenX.GT'],axis=1, inplace = True)
# df_crowd.drop(['pacbio.GT'],axis=1, inplace = True)
# df_crowd.drop(['IllMP.GT'],axis=1, inplace = True)
# df_crowd.drop(['Ill250.GT'],axis=1, inplace = True)
df_crowd.drop(['Ill300x.GT'],axis=1, inplace = True)
df_crowd.drop(['sample'],axis=1, inplace = True)
# df_crowd.columns.drop(['Label'],axis=1, inplace = True)
# df_crowd.drop(['size'],axis=1, inplace = True)
# df_crowd.drop(['Svsize'],axis=1, inplace = True)
# df_crowd.drop(['Ill250.amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)

In [16]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(d) - set(c)

{'GIAB_Crowd',
 'GTconflict',
 'GTcons',
 'GTsupp',
 'Ill300x.GT',
 'Ill300x.amb_reason_insertSizeScore_orientation',
 'SVtype',
 'Size',
 'chrom',
 'end',
 'id',
 'sample',
 'start',
 'type'}

In [23]:
### Prediction Set: Drop irrelevant columns and GT information
HG002_pred.drop(['GTcons'], axis=1, inplace = True)
HG002_pred.drop(['GTconflict'], axis=1, inplace = True)
HG002_pred.drop(['GTsupp'], axis=1, inplace = True)
HG002_pred.drop(['type'],axis=1, inplace = True)
HG002_pred.drop(['SVtype'],axis=1, inplace = True)
HG002_pred.drop(['start'],axis=1, inplace = True)
HG002_pred.drop(['end'],axis=1, inplace = True)
HG002_pred.drop(['chrom'],axis=1, inplace = True)
# HG002_pred.drop(['Size'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.GT'],axis=1, inplace = True)
# HG002_pred.drop(['pacbio.GT'],axis=1, inplace = True)
# HG002_pred.drop(['IllMP.GT'],axis=1, inplace = True)
# HG002_pred.drop(['Ill250.GT'],axis=1, inplace = True)
HG002_pred.drop(['Ill300x.GT'],axis=1, inplace = True)
HG002_pred.drop(['sample'],axis=1, inplace = True)
HG002_pred.drop(['id'],axis=1, inplace = True)
HG002_pred.drop(['Ill300x.amb_reason_insertSizeScore_orientation'],axis=1, inplace = True)
# HG002_pred.drop(['IllMP.amb_reason_insertSizeScore_alignmentScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP1_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP1_ref_reason_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP2_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP2_amb_reason_orientation_insertSizeScore'],axis=1, inplace = True)

In [24]:
#PB Dataframe contains missing values in the label column ['GIAB_Crowd'] - will drop these rows 
HG002_pred[HG002_pred['GIAB_Crowd'].notnull()]

,Size,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,...,Ill300x.ref_reason_alignmentScore,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-90,580.000000,18.547237,4.0,995.250000,257.372663,1.0,3.0,0.0,545.406928,...,186.0,0.0,1.0,0,0.0,1,1.000000,0,0,2
1,-33,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,539.468680,...,379.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
2,-145,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,541.033679,...,697.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
3,-98,568.000000,0.000000,1.0,1093.000000,0.000000,0.0,1.0,0.0,536.675808,...,445.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
4,-134,572.000000,0.000000,1.0,1038.000000,0.000000,0.0,1.0,0.0,526.852385,...,712.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
5,-22,584.000000,0.000000,1.0,1222.000000,0.000000,0.0,1.0,0.0,535.230132,...,341.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
6,-28,572.000000,0.000000,1.0,1187.000000,0.000000,0.0,1.0,0.0,546.830975,...,255.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
7,-231,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,520.882022,...,376.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
8,-172,581.928571,13.920510,14.0,1175.000000,129.746456,0.0,14.0,0.0,561.346408,...,472.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
9,-21,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,541.146597,...,317.0,0.0,0.0,0,0.0,1,1.000000,0,0,2


In [25]:
df_crowd.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,0.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,0.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,0.0,0.0,1329,3,0.059443,0,0.0,0,0,0


*** 
Impute missing values using KNN 
***

In [26]:
# Store data in a new variable which will be converted to a matrix
X = df_crowd
X.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,0.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,0.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,0.0,0.0,1329,3,0.059443,0,0.0,0,0,0


In [27]:
# Convert dataframe to matrix
X=X.as_matrix()

#Imput missing values from three closest observations
X_imputed=KNN(k=3).complete(X)
X=pd.DataFrame(X_imputed)

Imputing row 1/1515 with 0 missing, elapsed time: 0.610
Imputing row 101/1515 with 0 missing, elapsed time: 0.611
Imputing row 201/1515 with 0 missing, elapsed time: 0.729
Imputing row 301/1515 with 0 missing, elapsed time: 0.735
Imputing row 401/1515 with 0 missing, elapsed time: 0.741
Imputing row 501/1515 with 0 missing, elapsed time: 0.743
Imputing row 601/1515 with 0 missing, elapsed time: 0.745
Imputing row 701/1515 with 0 missing, elapsed time: 0.745
Imputing row 801/1515 with 0 missing, elapsed time: 0.747
Imputing row 901/1515 with 0 missing, elapsed time: 0.748
Imputing row 1001/1515 with 0 missing, elapsed time: 0.748
Imputing row 1101/1515 with 0 missing, elapsed time: 0.753
Imputing row 1201/1515 with 0 missing, elapsed time: 0.755
Imputing row 1301/1515 with 0 missing, elapsed time: 0.755
Imputing row 1401/1515 with 0 missing, elapsed time: 0.756
Imputing row 1501/1515 with 0 missing, elapsed time: 0.756


In [33]:
# Store header values in a list, will be used later to re-label the matrix post KNN imputation
dfcrowd_header = list(df_crowd.columns.values)
X.columns = dfcrowd_header
X.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,0.0,0.0,1651.0,8.0,0.096911,0.0,0.0,0.0,0.0,1.0
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,0.0,0.0,176.0,2.0,0.221591,0.0,0.0,0.0,0.0,1.0
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,0.0,0.0,1329.0,3.0,0.059443,0.0,0.0,0.0,0.0,0.0


In [34]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y['Label'] = X['Label']
#Y = X.pop('Label')

In [35]:
#Count the number of labels
pd.value_counts(Y['Label'].values, sort=False)

 1.0    959
 0.0    552
-1.0      1
 2.0      3
dtype: int64

In [36]:
# Remove labels from feature set
X.drop(['Label'],axis=1, inplace = True)

In [37]:
# Order features
X4 = X.reindex_axis(sorted(X.columns), axis=1)

***
Machine Learning
***

** Train RF Classifier **

In [38]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X4, Y, test_size=0.3, random_state=0)

In [39]:
# Train Random Forest Classifier
model = RandomForestClassifier() 
model.fit(X_train, y_train)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [40]:
#NOTE: Training Set - Show number of Hom Ref, Hom Var, Het Var datapoints the model was trained on
ytrain = pd.DataFrame()
ytrain['ytrain'] = y_train
pd.value_counts(ytrain['ytrain'].values, sort=False)

 1.0    671
 0.0     73
-1.0      1
 2.0      2
dtype: int64

** Precision Score **
- Overall model performance
- Using 30% of original dataset (test set)
- Truth labels: CrowdVariant labels

In [41]:
model.predict(X_test)

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,
        1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
        1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0

In [42]:
pred = model.predict(X_test)

In [43]:
precision_score(pred, y_test, average='micro') 

0.96703296703296704

In [44]:
# Add original labels and predicted labels back to the original dataframe
df_Xtest = pd.DataFrame(X_test)
df_Xtest.head()

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,Ill300x.ref_reason_alignmentScore,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,Size,refN_cnt,refN_pct,segdup_cnt,segdup_pct,tandemrep_cnt,tandemrep_pct
1222,589.435606,7.274880,528.0,633.607955,149.863131,243.0,285.0,0.0,541.900846,90.303799,...,0.0,0.0,0.0,320.0,0.0,0.0,0.0,0.000000,1.0,0.12500
310,584.644269,9.538767,253.0,635.561265,162.372800,127.0,126.0,0.0,529.934868,92.347999,...,281.0,0.0,1.0,320.0,0.0,0.0,0.0,0.000000,1.0,0.10625
9,567.346939,21.428144,49.0,716.285714,171.177077,0.0,30.0,19.0,549.786572,74.408848,...,364.0,0.0,42.0,233.0,0.0,0.0,0.0,0.000000,0.0,0.00000
785,581.916168,5.442583,167.0,599.826347,170.450582,162.0,5.0,0.0,532.700078,91.693132,...,329.0,0.0,0.0,121.0,0.0,0.0,1.0,0.404959,1.0,0.31405
295,577.156028,17.312843,141.0,609.879433,148.310508,74.0,67.0,0.0,520.960245,91.350233,...,175.0,0.0,0.0,324.0,0.0,0.0,0.0,0.000000,2.0,0.16358


In [45]:
labels = pd.DataFrame(y_test)

In [46]:
df_crowd.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,0.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,0.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,0.0,0.0,1329,3,0.059443,0,0.0,0,0,0


In [47]:
df_Xtest['predicted_label'] = pred
df_Xtest['Label'] = df_crowd_2['Label']
df_Xtest['chrom'] = df_crowd_2['chrom']
df_Xtest['start'] = df_crowd_2['start']
df_Xtest['end'] = df_crowd_2['end']
# df_Xtest['Y_test'] = labels

In [48]:
pd.value_counts(df_Xtest['Label'].values, sort=False)

0    166
1    288
2      1
dtype: int64

In [49]:
pd.value_counts(df_Xtest['predicted_label'].values, sort=False)

0.0    168
1.0    287
dtype: int64

In [50]:
from sklearn.metrics import confusion_matrix
ytest = df_Xtest['Label']
predict = df_Xtest['predicted_label']
print(confusion_matrix(ytest, predict))

[[160   6   0]
 [  8 280   0]
 [  0   1   0]]


In [51]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0,160,6,166
1,8,280,288
2,0,1,1
All,168,287,455


In [52]:
from sklearn.metrics import classification_report
print(classification_report(ytest, predict))

             precision    recall  f1-score   support

          0       0.95      0.96      0.96       166
          1       0.98      0.97      0.97       288
          2       0.00      0.00      0.00         1

avg / total       0.96      0.97      0.97       455



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



***
Predict
***

**Load Data**

In [53]:
HG002_pred.head(3)

,Size,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,...,Ill300x.ref_reason_alignmentScore,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-90,580.0,18.547237,4.0,995.25,257.372663,1.0,3.0,0.0,545.406928,...,186.0,0.0,1.0,0,0.0,1,1.0,0,0,2
1,-33,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,539.468680,...,379.0,0.0,0.0,0,0.0,0,0.0,0,0,2
2,-145,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,541.033679,...,697.0,0.0,0.0,0,0.0,0,0.0,0,0,2


In [54]:
X2 = HG002_pred

** Impute missing values using KNN **

In [55]:
#Convert dataframe to matrix
X2=X2.as_matrix()

#Imput missing values from three closest observations
X2_imputed=KNN(k=3).complete(X2)
X2=pd.DataFrame(X2_imputed)

Imputing row 1/2804 with 0 missing, elapsed time: 2.086
Imputing row 101/2804 with 0 missing, elapsed time: 2.087
Imputing row 201/2804 with 0 missing, elapsed time: 2.090
Imputing row 301/2804 with 0 missing, elapsed time: 2.090
Imputing row 401/2804 with 0 missing, elapsed time: 2.091
Imputing row 501/2804 with 0 missing, elapsed time: 2.091
Imputing row 601/2804 with 0 missing, elapsed time: 2.091
Imputing row 701/2804 with 0 missing, elapsed time: 2.092
Imputing row 801/2804 with 0 missing, elapsed time: 2.092
Imputing row 901/2804 with 0 missing, elapsed time: 2.093
Imputing row 1001/2804 with 0 missing, elapsed time: 2.093
Imputing row 1101/2804 with 0 missing, elapsed time: 2.096
Imputing row 1201/2804 with 0 missing, elapsed time: 2.096
Imputing row 1301/2804 with 0 missing, elapsed time: 2.098
Imputing row 1401/2804 with 0 missing, elapsed time: 2.098
Imputing row 1501/2804 with 0 missing, elapsed time: 2.098
Imputing row 1601/2804 with 0 missing, elapsed time: 2.099
Imputing 

In [56]:
dfcrowd_header = list(HG002_pred.columns.values)
X2.columns = dfcrowd_header
X2.head(3)

,Size,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,...,Ill300x.ref_reason_alignmentScore,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-90.0,580.0,18.547237,4.0,995.25,257.372663,1.0,3.0,0.0,545.406928,...,186.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
1,-33.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,539.468680,...,379.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,-145.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,541.033679,...,697.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [57]:
X3 = pd.DataFrame()
X3 = X2
X3.head(3)

,Size,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,...,Ill300x.ref_reason_alignmentScore,Ill300x.ref_reason_insertSizeScore,Ill300x.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-90.0,580.0,18.547237,4.0,995.25,257.372663,1.0,3.0,0.0,545.406928,...,186.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
1,-33.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,539.468680,...,379.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,-145.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,541.033679,...,697.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [58]:
X2.drop(['GIAB_Crowd'],axis=1, inplace = True)

In [59]:
# Order features
X5 = X2.reindex_axis(sorted(X2.columns), axis=1)

In [60]:
model.predict(X5)

array([ 1.,  1.,  1., ...,  0.,  0.,  0.])

In [61]:
df_new = pd.DataFrame()
df_new['true'] = HG002_pred['GIAB_Crowd'].astype(float)
df_new['predicted'] = model.predict(X5)
df_new['predicted'] = df_new['predicted'].astype(float)

In [62]:
pd.value_counts(df_new['true'].values, sort=False)

2.0     472
1.0    1734
0.0     598
dtype: int64

In [63]:
pd.value_counts(df_new['predicted'].values, sort=False)

1.0    2018
0.0     786
dtype: int64

In [64]:
# Step for processing missing values in label column
df_new_ = pd.DataFrame()
df_new_ = df_new[df_new['true'].notnull()]
df_new_

,true,predicted
0,2.0,1.0
1,2.0,1.0
2,2.0,1.0
3,2.0,1.0
4,2.0,1.0
5,2.0,1.0
6,2.0,1.0
7,2.0,1.0
8,2.0,1.0
9,2.0,1.0


In [65]:
df_new['true']

0       2.0
1       2.0
2       2.0
3       2.0
4       2.0
5       2.0
6       2.0
7       2.0
8       2.0
9       2.0
10      2.0
11      2.0
12      2.0
13      2.0
14      2.0
15      2.0
16      2.0
17      2.0
18      2.0
19      2.0
20      2.0
21      2.0
22      2.0
23      2.0
24      2.0
25      2.0
26      2.0
27      2.0
28      2.0
29      2.0
       ... 
2774    0.0
2775    0.0
2776    0.0
2777    0.0
2778    0.0
2779    0.0
2780    0.0
2781    0.0
2782    0.0
2783    0.0
2784    0.0
2785    0.0
2786    0.0
2787    0.0
2788    0.0
2789    0.0
2790    0.0
2791    0.0
2792    0.0
2793    0.0
2794    0.0
2795    0.0
2796    0.0
2797    0.0
2798    0.0
2799    0.0
2800    0.0
2801    0.0
2802    0.0
2803    0.0
Name: true, dtype: float64

In [66]:
# Calculate Precision Score
true = df_new_['true']
predicted = df_new_['predicted']
precision_score(true, predicted, average='micro') 

0.76925820256776034

In [67]:
pd.value_counts(df_new['true'].values, sort=False)

2.0     472
1.0    1734
0.0     598
dtype: int64

In [68]:
pd.crosstab(true, predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0.0,597,1,598
1.0,174,1560,1734
2.0,15,457,472
All,786,2018,2804


In [69]:
from sklearn.metrics import classification_report
true = df_new_['true']
pred = df_new_['predicted']
print(classification_report(true, pred))

             precision    recall  f1-score   support

        0.0       0.76      1.00      0.86       598
        1.0       0.77      0.90      0.83      1734
        2.0       0.00      0.00      0.00       472

avg / total       0.64      0.77      0.70      2804



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [70]:
# Store Chr X and Y as numbers (23, 24)
X5['chrom'].replace('X', 23, inplace=True)
X5['chrom'].replace('Y', 24, inplace=True)

KeyError: 'chrom'

In [71]:
pred_prob = model.predict_proba(X5)

In [72]:
X5['predicted_label'] = pred
X5['Label'] = HG002_pred_2['GIAB_Crowd']
X5['chrom'] = HG002_pred_2['chrom']
X5['start'] = HG002_pred_2['start']
X5['end'] = HG002_pred_2['end']

In [73]:
X6 = pd.concat([X5, pd.DataFrame(pred_prob, columns=['1','2','3','4'])])

In [74]:
X6.to_csv('300x_pred_prob.csv', index=False)

In [75]:
X6 = pd.read_csv('300x_pred_prob.csv')

In [76]:
X6.rename(columns={'1': 'unknown'}, inplace=True)
X6.rename(columns={'2': 'Hom_Var'}, inplace=True)
X6.rename(columns={'3': 'Het_Var'}, inplace=True)
X6.rename(columns={'4': 'Hom_Ref'}, inplace=True)

** Show Size Distribution of Labels with Predict Probability > 0.8 **

In [77]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)

In [78]:
X_homVar_conf = X6[(X6['Hom_Var'] == 0.9) | (X6['Hom_Var'] == 1) | (X6['Hom_Var'] == 0.8)]

In [79]:
X_homVar_conf['Size'] = HG002_pred_2['Size'].abs()

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [80]:
output_notebook()

Loading BokehJS ...

In [81]:
# all_df_del['abs_size'] = all_df_del['Size'].abs()
X_homVar_conf['Size_Log'] = np.log10(X_homVar_conf.Size)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [82]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_conf, values='Size_Log', title='Homozygous Variant: Predict Proba > 0.8', color='LightSlateGray', bins=30, xlabel="Size(Log10)", ylabel="Frequency")
output_file("PB_HomVar_PredProb0.8+.html")
show(p)

![Figure 2](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/PB_HomVar_PredProb0.8%2B_.png)

![Figure 2](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/PB_HomVar_PredProb0.8%2B_.png)

In [ ]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_conf = X6[(X6['Het_Var'] == 0.9) | (X6['Het_Var'] == 1) | (X6['Het_Var'] == 0.8)]
X_hetVar_conf['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_conf['Size_Log'] = np.log10(X_hetVar_conf.Size)

In [ ]:
output_notebook()

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_conf, values='Size_Log', title='Heterozygous Variant: Predict Proba > 0.9', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("PB_HetVar_PredProb0.9+.html")
show(p)

![Figure 2](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/PB_HetVar_PredProb0.9%2B.png)

** Show Size Distribution of Labels with Predict Probability < 0.3 **

In [ ]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)
X_homVar_least = X6[(X6['Hom_Var'] == 0.3) | (X6['Hom_Var'] == 0.2) | (X6['Hom_Var'] == 0.1)]
X_homVar_least['Size'] = HG002_pred_2['Size'].abs()
X_homVar_least['Size_Log'] = np.log10(X_homVar_least.Size)

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_least, values='Size_Log', title='Homozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("PB_HomVar_PredProb0.3-.html")
show(p)

![Figure 1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/PB_HomVar_PredProb0.3-.png)

In [ ]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_least = X6[(X6['Het_Var'] == 0.3) | (X6['Het_Var'] == 0.2) | (X6['Het_Var'] == 0.1)]
X_hetVar_least['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_least['Size_Log'] = np.log10(X_hetVar_least.Size)

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_least, values='Size_Log', title='Heterozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("PB_HetVar_PredProb0.3-.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/PB_HetVar_PredProb0.3-.png)

** Size Distribution of Original Data **

In [ ]:
p = figure()
p = Histogram(df_crowd_2, values='size', title='CrowdVariant Data Size Distribution', color='LightSlateGray', bins=30, xlabel="Size(bp)", ylabel="Frequency")
output_file("CrowdVar.Size.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/crowdVar_size.png)

In [ ]:
del_data = pd.read_csv('/Users/lmc2/Desktop/log_Size_.csv')

In [ ]:
p = figure()
p = Histogram(del_data, values='DEL_log_size', title='SVVIZ HG002 Rand Selection Data Size Distribution', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("SVVIZ.Size.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/svviz_size.png)

**TODO**

[Feature Importance](http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)